In [ ]:
from textblob import TextBlob
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
import string
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import plotly.express as px
import pandas as pd

In [ ]:
df = pd.read_pickle('./data_speech.pickle')
data = pd.DataFrame(df)
data = data [['index', 'title','date', 'content']]

In [ ]:
obj = 1
tar = data.content[0]
t_tot = [s for s in tar if len(s) > 30]

In [ ]:

tweet_list = pd.DataFrame(t_tot)
tweet_list.drop_duplicates(inplace=True)

print('Tweets a analizar: ',len(tweet_list))


In [ ]:
tweet_list['text'] = tweet_list[0]
traducir = lambda x: TextBlob(x).translate(to="en")
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|([\t])|(\w+:\/\/\S+)","  ",x)
tweet_list["text"] = tweet_list.text.map(remove_rt).map(rt)
tweet_list["text"] = tweet_list.text.str.lower()
tweet_list['text'] = tweet_list['text'].apply(traducir)
tweet_list['text'] = tweet_list['text'].apply(lambda x: ''.join(x))
tweet_list['text'].head(10)

In [ ]:
tweet_list[['polarity', 'subjectivity']] = tweet_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tweet_list['text'].iteritems():
 score = SentimentIntensityAnalyzer().polarity_scores(row)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']
 if neg > pos:
    tweet_list.loc[index, 'sentiment'] = "negative"
 elif pos > neg:
    tweet_list.loc[index, 'sentiment'] = "positive"
 else:
    tweet_list.loc[index, 'sentiment'] = "neutral"
    tweet_list.loc[index, 'neg'] = neg
    tweet_list.loc[index, 'neu'] = neu
    tweet_list.loc[index, 'pos'] = pos
    tweet_list.loc[index, 'compound'] = comp
tweet_list[['text','polarity','subjectivity','sentiment']].head(10)

In [ ]:
tweet_list_negative = tweet_list[tweet_list["sentiment"]=="negative"]
tweet_list_positive = tweet_list[tweet_list["sentiment"]=="positive"]
tweet_list_neutral = tweet_list[tweet_list["sentiment"]=="neutral"]

In [ ]:
def count_values_in_column(data,feature):
 total=data.loc[:,feature].value_counts(dropna=False)
 percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
 return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [ ]:
res = count_values_in_column(tweet_list,"sentiment")
res

In [ ]:
res.index.to_list()

In [ ]:
sizes = res['Total'].to_list()
names = res.index.to_list()
title = 'Análisis de sentimiento para discurso ' + str(1)

In [ ]:
fig = px.pie(res, values=sizes, names=names, title=title ,color_discrete_sequence= px.colors.qualitative.Safe)
fig.show()

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('spanish'))

wordcloud = WordCloud(stopwords=stop_words,max_words=3000 ,background_color="white", repeat=False).generate(str(tweet_list["text"].values))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.figure(figsize=(40,40))
plt.show(sizes)

In [ ]:
wordcloud = WordCloud(stopwords=stop_words,max_words=3000 ,background_color="white", repeat=False).generate(str(tweet_list_positive["text"].values))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(stopwords=stop_words,max_words=3000 ,background_color="white", repeat=False).generate(str(tweet_list_negative["text"].values))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(stopwords=stop_words,max_words=3000 ,background_color="white", repeat=False).generate(str(tweet_list_neutral["text"].values))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()